<a href="https://colab.research.google.com/github/PhoenixStormJr/RVC-v2-easy-GUI-glitches-fixed/blob/main/UVR5_NO_UI_Prepare_RVC_dataset__3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **🎵 UVR5 NO UI 🎵**

Colab created by **[Not Eddy (Spanish Mod)](http://discord.com/users/274566299349155851)** in **[AI HUB](https://discord.gg/aihub)** community.

<br>**This is a CLI type colab created based on the [beveradb's python-audio-separator](https://github.com/karaokenerds/python-audio-separator) repository.<br>
If you liked this colab you can star my repo on [GitHub](https://github.com/Eddycrack864/UVR5-NO-UI)**.<br>

<br>You can donate to the original UVR5 project here:<br>
[!["Buy Me A Coffee"](https://www.buymeacoffee.com/assets/img/custom_images/orange_img.png)](https://www.buymeacoffee.com/uvr5)

<br>**How To Use: https://github.com/Eddycrack864/UVR5-NO-UI#how-to-use**

![](https://count.nett.moe/get/uvr5_no_ui_colab/img?theme=rule34)

In [ ]:
#@title # **Installation**
#@markdown ####This takes around 2 minutes
from IPython.display import clear_output
import subprocess
import os

colab_path = "/content"
kaggle_path = "/kaggle/working"

if os.path.exists(colab_path):
  print("Welcome to Colab Notebook")
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)
  path = "/content"
elif os.path.exists(kaggle_path):
  print("Welcome to Kaggle Notebook")
  path = "/kaggle/working"

!pip install "audio-separator[gpu]==0.32.0"
subprocess.run(["pip", "install", "demucs"])
!pip install aria2
!pip install yt_dlp
!mkdir models
!mkdir temp
!aria2c https://huggingface.co/Eddycrack864/Drumsep/resolve/main/modelo_final.th -o models/drumsep.th
!apt-get update
!apt-get install -y libcudnn8

clear_output()
print('Installation done !')

In [ ]:
#@title ### ***Get Vocals ONLY!***
#@markdown #**Separation! MelBand Roformer and MDX-NET**
import os
import glob
import yt_dlp
from google.colab import files


###################################################################################################
###################################################################################################
#FUNCTIONS:
###################################################################################################
###################################################################################################
def downloader(url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
        'outtmpl': os.path.join(f'{path}/temp', '%(title)s.%(ext)s'),
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

def checker(url):
    return "http" in url

def uvr_cli(input_folder, output_folder, model, output_format, segment_size, overlap, batch_size, override_model_segment_size, use_autocast, extensions):
    found_files = []
    dictmodel = {
      'BS-Roformer-Viperx-1297': 'model_bs_roformer_ep_317_sdr_12.9755.ckpt',
      'BS-Roformer-Viperx-1296': 'model_bs_roformer_ep_368_sdr_12.9628.ckpt',
      'BS-Roformer-Viperx-1053': 'model_bs_roformer_ep_937_sdr_10.5309.ckpt',
      'Mel-Roformer-Viperx-1143': 'model_mel_band_roformer_ep_3005_sdr_11.4360.ckpt',
      'BS-Roformer-De-Reverb': 'deverb_bs_roformer_8_384dim_10depth.ckpt',
      'Mel-Roformer-Crowd-Aufr33-Viperx': 'mel_band_roformer_crowd_aufr33_viperx_sdr_8.7144.ckpt',
      'Mel-Roformer-Denoise-Aufr33': 'denoise_mel_band_roformer_aufr33_sdr_27.9959.ckpt',
      'Mel-Roformer-Denoise-Aufr33-Aggr' : 'denoise_mel_band_roformer_aufr33_aggr_sdr_27.9768.ckpt',
      'MelBand Roformer | Denoise-Debleed by Gabox' : 'mel_band_roformer_denoise_debleed_gabox.ckpt',
      'Mel-Roformer-Karaoke-Aufr33-Viperx': 'mel_band_roformer_karaoke_aufr33_viperx_sdr_10.1956.ckpt',
      'MelBand Roformer | Karaoke by Gabox' : 'mel_band_roformer_karaoke_gabox.ckpt',
      'MelBand Roformer | Karaoke by becruily' : 'mel_band_roformer_karaoke_becruily.ckpt',
      'MelBand Roformer | Vocals by Kimberley Jensen' : 'vocals_mel_band_roformer.ckpt',
      'MelBand Roformer Kim | FT by unwa' : 'mel_band_roformer_kim_ft_unwa.ckpt',
      'MelBand Roformer Kim | FT 2 by unwa' : 'mel_band_roformer_kim_ft2_unwa.ckpt',
      'MelBand Roformer Kim | FT 2 Bleedless by unwa' : 'mel_band_roformer_kim_ft2_bleedless_unwa.ckpt',
      'MelBand Roformer Kim | FT 3 by unwa' : 'mel_band_roformer_kim_ft3_unwa.ckpt',
      'MelBand Roformer Kim | Inst V1 by Unwa' : 'melband_roformer_inst_v1.ckpt',
      'MelBand Roformer Kim | Inst V1 Plus by Unwa' : 'melband_roformer_inst_v1_plus.ckpt',
      'MelBand Roformer Kim | Inst V1 (E) by Unwa' : 'melband_roformer_inst_v1e.ckpt',
      'MelBand Roformer Kim | Inst V1 (E) Plus by Unwa' : 'melband_roformer_inst_v1e_plus.ckpt',
      'MelBand Roformer Kim | Inst V2 by Unwa' : 'melband_roformer_inst_v2.ckpt',
      'MelBand Roformer Kim | InstVoc Duality V1 by Unwa' : 'melband_roformer_instvoc_duality_v1.ckpt',
      'MelBand Roformer Kim | InstVoc Duality V2 by Unwa' : 'melband_roformer_instvox_duality_v2.ckpt',
      'MelBand Roformer | Vocals by becruily' : 'mel_band_roformer_vocals_becruily.ckpt',
      'MelBand Roformer | Instrumental by becruily' : 'mel_band_roformer_instrumental_becruily.ckpt',
      'MelBand Roformer | Vocals Fullness by Aname' : 'mel_band_roformer_vocal_fullness_aname.ckpt',
      'BS Roformer | Vocals by Gabox' : 'bs_roformer_vocals_gabox.ckpt',
      'MelBand Roformer | Vocals by Gabox' : 'mel_band_roformer_vocals_gabox.ckpt',
      'MelBand Roformer | Vocals FV1 by Gabox' : 'mel_band_roformer_vocals_fv1_gabox.ckpt',
      'MelBand Roformer | Vocals FV2 by Gabox' : 'mel_band_roformer_vocals_fv2_gabox.ckpt',
      'MelBand Roformer | Vocals FV3 by Gabox' : 'mel_band_roformer_vocals_fv3_gabox.ckpt',
      'MelBand Roformer | Vocals FV4 by Gabox' : 'mel_band_roformer_vocals_fv4_gabox.ckpt',
      'MelBand Roformer | Instrumental by Gabox' : 'mel_band_roformer_instrumental_gabox.ckpt',
      'MelBand Roformer | Instrumental 2 by Gabox' : 'mel_band_roformer_instrumental_2_gabox.ckpt',
      'MelBand Roformer | Instrumental 3 by Gabox' : 'mel_band_roformer_instrumental_3_gabox.ckpt',
      'MelBand Roformer | Instrumental Bleedless V1 by Gabox' : 'mel_band_roformer_instrumental_bleedless_v1_gabox.ckpt',
      'MelBand Roformer | Instrumental Bleedless V2 by Gabox' : 'mel_band_roformer_instrumental_bleedless_v2_gabox.ckpt',
      'MelBand Roformer | Instrumental Bleedless V3 by Gabox' : 'mel_band_roformer_instrumental_bleedless_v3_gabox.ckpt',
      'MelBand Roformer | Instrumental Fullness V1 by Gabox' : 'mel_band_roformer_instrumental_fullness_v1_gabox.ckpt',
      'MelBand Roformer | Instrumental Fullness V2 by Gabox' : 'mel_band_roformer_instrumental_fullness_v2_gabox.ckpt',
      'MelBand Roformer | Instrumental Fullness V3 by Gabox' : 'mel_band_roformer_instrumental_fullness_v3_gabox.ckpt',
      'MelBand Roformer | Instrumental Fullness Noisy V4 by Gabox' : 'mel_band_roformer_instrumental_fullness_noise_v4_gabox.ckpt',
      'MelBand Roformer | INSTV5 by Gabox' : 'mel_band_roformer_instrumental_instv5_gabox.ckpt',
      'MelBand Roformer | INSTV5N by Gabox' : 'mel_band_roformer_instrumental_instv5n_gabox.ckpt',
      'MelBand Roformer | INSTV6 by Gabox' : 'mel_band_roformer_instrumental_instv6_gabox.ckpt',
      'MelBand Roformer | INSTV6N by Gabox' : 'mel_band_roformer_instrumental_instv6n_gabox.ckpt',
      'MelBand Roformer | INSTV7 by Gabox' : 'mel_band_roformer_instrumental_instv7_gabox.ckpt',
      'MelBand Roformer | INSTV7N by Gabox' : 'mel_band_roformer_instrumental_instv7n_gabox.ckpt',
      'MelBand Roformer | INSTV8 by Gabox' : 'mel_band_roformer_instrumental_instv8_gabox.ckpt',
      'MelBand Roformer | INSTV8N by Gabox' : 'mel_band_roformer_instrumental_instv8n_gabox.ckpt',
      'MelBand Roformer | FVX by Gabox' : 'mel_band_roformer_instrumental_fvx_gabox.ckpt',
      'MelBand Roformer | De-Reverb by anvuew' : 'dereverb_mel_band_roformer_anvuew_sdr_19.1729.ckpt',
      'MelBand Roformer | De-Reverb Less Aggressive by anvuew' : 'dereverb_mel_band_roformer_less_aggressive_anvuew_sdr_18.8050.ckpt',
      'MelBand Roformer | De-Reverb Mono by anvuew' : 'dereverb_mel_band_roformer_mono_anvuew.ckpt',
      'MelBand Roformer | De-Reverb Big by Sucial' : 'dereverb_big_mbr_ep_362.ckpt',
      'MelBand Roformer | De-Reverb Super Big by Sucial' : 'dereverb_super_big_mbr_ep_346.ckpt',
      'MelBand Roformer | De-Reverb-Echo by Sucial' : 'dereverb-echo_mel_band_roformer_sdr_10.0169.ckpt',
      'MelBand Roformer | De-Reverb-Echo V2 by Sucial' : 'dereverb-echo_mel_band_roformer_sdr_13.4843_v2.ckpt',
      'MelBand Roformer | De-Reverb-Echo Fused by Sucial' : 'dereverb_echo_mbr_fused.ckpt',
      'MelBand Roformer Kim | SYHFT by SYH99999' : 'MelBandRoformerSYHFT.ckpt',
      'MelBand Roformer Kim | SYHFT V2 by SYH99999' : 'MelBandRoformerSYHFTV2.ckpt',
      'MelBand Roformer Kim | SYHFT V2.5 by SYH99999' : 'MelBandRoformerSYHFTV2.5.ckpt',
      'MelBand Roformer Kim | SYHFT V3 by SYH99999' : 'MelBandRoformerSYHFTV3Epsilon.ckpt',
      'MelBand Roformer Kim | Big SYHFT V1 by SYH99999' : 'MelBandRoformerBigSYHFTV1.ckpt',
      'MelBand Roformer Kim | Big Beta 4 FT by unwa' : 'melband_roformer_big_beta4.ckpt',
      'MelBand Roformer Kim | Big Beta 5e FT by unwa' : 'melband_roformer_big_beta5e.ckpt',
      'MelBand Roformer | Big Beta 6 by unwa' : 'melband_roformer_big_beta6.ckpt',
      'MelBand Roformer | Big Beta 6X by unwa' : 'melband_roformer_big_beta6x.ckpt',
      'BS Roformer | Chorus Male-Female by Sucial' : 'model_chorus_bs_roformer_ep_267_sdr_24.1275.ckpt',
      'BS Roformer | Male-Female by aufr33' : 'bs_roformer_male_female_by_aufr33_sdr_7.2889.ckpt',
      'MelBand Roformer | Aspiration by Sucial' : 'aspiration_mel_band_roformer_sdr_18.9845.ckpt',
      'MelBand Roformer | Aspiration Less Aggressive by Sucial' : 'aspiration_mel_band_roformer_less_aggr_sdr_18.1201.ckpt',
      'MelBand Roformer | Bleed Suppressor V1 by unwa-97chris' : 'mel_band_roformer_bleed_suppressor_v1.ckpt'
    }
    roformer_model = dictmodel[model]
    if not os.path.exists(input_folder):
        os.makedirs(input_folder)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    if checker(input_folder):
        downloader(input_folder)
        input_folder = f"{path}/temp"
    for audio_files in os.listdir(input_folder):
        if audio_files.endswith(extensions):
            found_files.append(audio_files)
    if len(found_files) == 0:
        print(f"No valid audio files found in the directory \"{input_folder}\". Please upload them:")
        while len(found_files) == 0:
            uploaded = files.upload()
            for audio_files in os.listdir(input_folder):
                if audio_files.endswith(extensions):
                    found_files.append(audio_files)
    if len(found_files) > 0:
        print(f"{len(found_files)} audio files found")
        found_files.sort()
        for audio_files in found_files:
            file_path = os.path.join(input_folder, audio_files)
            prompt = f'audio-separator "{file_path}" --model_filename {roformer_model} --output_dir={output_folder} --output_format={output_format} --mdxc_segment_size={segment_size} --mdxc_overlap={overlap} --mdxc_batch_size={batch_size} --model_file_dir=./models'
            if override_model_segment_size:
                prompt += " --mdxc_override_model_segment_size"
            if use_autocast:
                prompt += " --use_autocast"
            !$prompt

def uvr_cli_MDX_NET(input_folder, output_folder, extensions, output_format, model, hop_length, segment_size, denoise, overlap, batch_size, use_autocast):
    found_files = []
    if checker(input_folder):
        downloader(input_folder)
        input_folder = f"{path}/temp"
    for audio_files in os.listdir(input_folder):
        if audio_files.endswith(extensions):
            found_files.append(audio_files)
    total_files = len(found_files)
    if total_files == 0:
        print("No valid audio files found.")
    else:
        print(f"{total_files} audio files found")
        found_files.sort()
        for audio_files in found_files:
            file_path = os.path.join(input_folder, audio_files)
            prompt = f'audio-separator "{file_path}" --model_filename {model} --output_dir={output_folder} --output_format={output_format} --mdx_hop_length={hop_length} --mdx_segment_size={segment_size} --mdx_overlap={overlap} --mdx_batch_size={batch_size} --model_file_dir=./models'
            if denoise:
                prompt += " --mdx_enable_denoise"
            if use_autocast:
                prompt += " --use_autocast"
            !$prompt
###################################################################################################
###################################################################################################
#END OF FUNCTIONS
###################################################################################################
###################################################################################################



#@markdown Input path for audio files or link:
input_folder = "/content/drive/MyDrive/audio_input" #@param {type:"string"}
#@markdown You can paste the link to the video/audio from many sites, check the complete list [here](https://github.com/yt-dlp/yt-dlp/blob/master/supportedsites.md)
#@markdown Output path for audio files:
output_folder = "/content/drive/MyDrive/audio_output" #@param {type:"string"}
#@markdown Select the output format:
output_format = "wav" #@param ["wav", "flac", "mp3", "ogg", "opus", "m4a", "aiff", "ac3"]
#@markdown Larger consumes more resources, but may give better results.
segment_size = 4000 #@param {type:"slider", min:32, max:4000, step:32}
#@markdown Amount of overlap between prediction windows.
overlap = 8 #@param {type:"slider", min:2, max:10, step:1}
#@markdown Larger consumes more RAM but may process slightly faster.
batch_size = 1 #@param {type:"slider", min:1, max:16, step:1}
#@markdown Override model default segment size instead of using the model default value.
override_model_segment_size = False #@param {type:"boolean"}
#@markdown Flag to use PyTorch autocast for faster inference. Do not use for CPU inference.
use_autocast = True #@param {type:"boolean"}
#@markdown ---
#@markdown ---
#@markdown ---
#@markdown Usually called stride in neural networks; only change if you know what you're doing.
hop_length = 1024 #@param {type:"slider", min:32, max:2048, step:32}
#@markdown Amount of overlap between prediction windows.
overlap_2 = "0.25" #@param ["0.25", "0.5", "0.75", "0.99"]
#@markdown Enable denoising during separation.
denoise = True #@param {type:"boolean"}
#@markdown ---
#@markdown ---
#@markdown ---
#@markdown Which models would you like to use?:
#@markdown
#@markdown MelBand Roformer models:
Vocals_by_Gabox = True #@param {type:"boolean"}
De_Reverb_Echo_Fused_by_Sucial = True #@param {type:"boolean"}
Denoise_Debleed_by_Gabox = True #@param {type:"boolean"}
Vocals_FV2_by_Gabox = True #@param {type:"boolean"}
#@markdown ---
#@markdown MDX-NET models:
kuielab_a_vocals_onnx = True #@param {type:"boolean"}
kuielab_b_vocals_onnx = True #@param {type:"boolean"}
#@markdown ---
#@markdown EXTREME models (Severely remove sound effects):
Final_Aggressive_Mode = "UVR-MDX-NET_Crowd_HQ_1.onnx (EXTREMELY AGGRESSIVE)" #@param ["MelBand Roformer | Karaoke by Gabox (Normal Aggression)", "UVR-MDX-NET_Crowd_HQ_1.onnx (EXTREMELY AGGRESSIVE)"]
Remove_Hissing_Level = "MelBand Roformer | Aspiration by Sucial" #@param ["MelBand Roformer | Aspiration by Sucial" , "MelBand Roformer | Aspiration Less Aggressive by Sucial"]



extensions = (".wav", ".flac", ".mp3", ".ogg", ".opus", ".m4a", ".aiff", ".ac3")

if not os.path.exists(input_folder):
    os.makedirs(input_folder)
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
found_files_1 = []
for audio_files in os.listdir(input_folder):
    if audio_files.endswith(extensions):
        found_files_1.append(audio_files)
Name_Of_Audio = found_files_1[0]
Name_Of_Audio_Fixed = found_files_1[0].replace(".","_fixed.")
Name_Of_Audio_Fixed_Aggressive = found_files_1[0].replace(".","_fixed_aggressive.")
Name_Of_Audio_Fixed_NoHiss = found_files_1[0].replace(".","_fixed_NoHiss.")
Name_Of_Audio_Fixed_Aggressive_NoHiss = found_files_1[0].replace(".","_fixed_aggressive_NoHiss.")

Name_Of_Audio_Fixed_EXTREMELYAGGRESSIVE_NoHiss = found_files_1[0].replace(".","_fixed_EXTREMELYAGGRESSIVE_NoHiss.")
Name_Of_Audio_Fixed_EXTREMELYAGGRESSIVE = found_files_1[0].replace(".","_fixed_EXTREMELYAGGRESSIVE.")


FolderNumber = 1



if Vocals_by_Gabox:
  if os.path.exists(f"/content/{str(FolderNumber)}"):
    !rm -rf "/content/{FolderNumber}"
  uvr_cli(input_folder, f"/content/{str(FolderNumber)}", "MelBand Roformer | Vocals by Gabox", output_format, segment_size, overlap, batch_size, override_model_segment_size, use_autocast, extensions)
  for file1 in os.listdir(f"/content/{str(FolderNumber)}"):
    if("(Instrumental)_mel_band_roformer_vocals_gabox" in file1):
      print(f"removing: {file1}")
      !rm "/content/{FolderNumber}/{file1}"
  FolderNumber = FolderNumber + 1
  FolderNumberPrevious = FolderNumber - 1
  for file1 in os.listdir(f"/content/{str(FolderNumberPrevious)}"):
    !mv "/content/{FolderNumberPrevious}/{file1}" "/content/{FolderNumberPrevious}/{Name_Of_Audio}"


if De_Reverb_Echo_Fused_by_Sucial:
  if os.path.exists(f"/content/{str(FolderNumber)}"):
    !rm -rf "/content/{FolderNumber}"
  uvr_cli(f"/content/{str(FolderNumber-1)}", f"/content/{str(FolderNumber)}", "MelBand Roformer | De-Reverb-Echo Fused by Sucial", output_format, segment_size, overlap, batch_size, override_model_segment_size, use_autocast, extensions)
  for file1 in os.listdir(f"/content/{str(FolderNumber)}"):
    if("(other)_dereverb_echo_mbr_fused" in file1):
      print(f"removing: {file1}")
      !rm "/content/{FolderNumber}/{file1}"
  FolderNumber = FolderNumber + 1
  FolderNumberPrevious = FolderNumber - 1
  for file1 in os.listdir(f"/content/{str(FolderNumberPrevious)}"):
    !mv "/content/{FolderNumberPrevious}/{file1}" "/content/{FolderNumberPrevious}/{Name_Of_Audio}"


if Denoise_Debleed_by_Gabox:
  if os.path.exists(f"/content/{str(FolderNumber)}"):
    !rm -rf "/content/{FolderNumber}"
  uvr_cli(f"/content/{str(FolderNumber-1)}", f"/content/{str(FolderNumber)}", "MelBand Roformer | Denoise-Debleed by Gabox", output_format, segment_size, overlap, batch_size, override_model_segment_size, use_autocast, extensions)
  for file1 in os.listdir(f"/content/{str(FolderNumber)}"):
    if("(Vocals)_mel_band_roformer_denoise_debleed_gabox" in file1):
      print(f"removing: {file1}")
      !rm "/content/{FolderNumber}/{file1}"
  FolderNumber = FolderNumber + 1
  FolderNumberPrevious = FolderNumber - 1
  for file1 in os.listdir(f"/content/{str(FolderNumberPrevious)}"):
    !mv "/content/{FolderNumberPrevious}/{file1}" "/content/{FolderNumberPrevious}/{Name_Of_Audio}"


if Vocals_FV2_by_Gabox:
  if os.path.exists(f"/content/{str(FolderNumber)}"):
    !rm -rf "/content/{FolderNumber}"
  uvr_cli(f"/content/{str(FolderNumber-1)}", f"/content/{str(FolderNumber)}", "MelBand Roformer | Vocals FV2 by Gabox", output_format, segment_size, overlap, batch_size, override_model_segment_size, use_autocast, extensions)
  for file1 in os.listdir(f"/content/{str(FolderNumber)}"):
    if("(Instrumental)_mel_band_roformer_vocals_fv2_gabox" in file1):
      print(f"removing: {file1}")
      !rm "/content/{FolderNumber}/{file1}"
  FolderNumber = FolderNumber + 1
  FolderNumberPrevious = FolderNumber - 1
  for file1 in os.listdir(f"/content/{str(FolderNumberPrevious)}"):
    !mv "/content/{FolderNumberPrevious}/{file1}" "/content/{FolderNumberPrevious}/{Name_Of_Audio}"


if kuielab_a_vocals_onnx:
  if os.path.exists(f"/content/{str(FolderNumber)}"):
    !rm -rf "/content/{FolderNumber}"
  uvr_cli_MDX_NET(f"/content/{str(FolderNumber-1)}", f"/content/{str(FolderNumber)}", extensions, output_format, "kuielab_a_vocals.onnx", hop_length, segment_size, denoise, overlap_2, batch_size, use_autocast)
  for file1 in os.listdir(f"/content/{str(FolderNumber)}"):
    if("(Instrumental)_kuielab_a_vocals" in file1):
      print(f"removing: {file1}")
      !rm "/content/{FolderNumber}/{file1}"
  FolderNumber = FolderNumber + 1
  FolderNumberPrevious = FolderNumber - 1
  for file1 in os.listdir(f"/content/{str(FolderNumberPrevious)}"):
    !mv "/content/{FolderNumberPrevious}/{file1}" "/content/{FolderNumberPrevious}/{Name_Of_Audio}"


if kuielab_b_vocals_onnx:
  if os.path.exists(f"/content/{str(FolderNumber)}"):
    !rm -rf "/content/{FolderNumber}"
  uvr_cli_MDX_NET(f"/content/{str(FolderNumber-1)}", f"/content/{str(FolderNumber)}", extensions, output_format, "kuielab_b_vocals.onnx", hop_length, segment_size, denoise, overlap_2, batch_size, use_autocast)
  for file1 in os.listdir(f"/content/{str(FolderNumber)}"):
    if("(Instrumental)_kuielab_b_vocals" in file1):
      print(f"removing: {file1}")
      !rm "/content/{FolderNumber}/{file1}"
  FolderNumber = FolderNumber + 1
  FolderNumberPrevious = FolderNumber - 1
  for file1 in os.listdir(f"/content/{str(FolderNumberPrevious)}"):
    !mv "/content/{FolderNumberPrevious}/{file1}" "/content/{FolderNumberPrevious}/{Name_Of_Audio}"

#Saves the result without the aggressive parts:
if not os.path.exists(output_folder):
  os.makedirs(output_folder)
FolderNumberPrevious = FolderNumber - 1
for file1 in os.listdir(f"/content/{str(FolderNumberPrevious)}"):
  !cp "/content/{FolderNumberPrevious}/{file1}" "{output_folder}/{Name_Of_Audio_Fixed}"



#Saves the result and removes the hissing:
if Remove_Hissing_Level == "MelBand Roformer | Aspiration Less Aggressive by Sucial":
  if os.path.exists(f"/content/{str(FolderNumber)}"):
    !rm -rf "/content/{FolderNumber}"
  uvr_cli(f"/content/{str(FolderNumber-1)}", f"/content/{str(FolderNumber)}", "MelBand Roformer | Aspiration Less Aggressive by Sucial", output_format, segment_size, overlap, batch_size, override_model_segment_size, use_autocast, extensions)
  for file1 in os.listdir(f"/content/{str(FolderNumber)}"):
    if("(aspiration)_aspiration_mel_band_roformer_less_aggr_sdr_18" in file1):
      print(f"removing: {file1}")
      !rm "/content/{FolderNumber}/{file1}"
  FolderNumber = FolderNumber + 1
  FolderNumberPrevious = FolderNumber - 1
  for file1 in os.listdir(f"/content/{str(FolderNumberPrevious)}"):
    !mv "/content/{FolderNumberPrevious}/{file1}" "/content/{FolderNumberPrevious}/{Name_Of_Audio}"
  if not os.path.exists(output_folder):
    os.makedirs(output_folder)
  FolderNumberPrevious = FolderNumber - 1
  for file1 in os.listdir(f"/content/{str(FolderNumberPrevious)}"):
    !cp "/content/{FolderNumberPrevious}/{file1}" "{output_folder}/{Name_Of_Audio_Fixed_NoHiss}"

if Remove_Hissing_Level == "MelBand Roformer | Aspiration by Sucial":
  if os.path.exists(f"/content/{str(FolderNumber)}"):
    !rm -rf "/content/{FolderNumber}"
  uvr_cli(f"/content/{str(FolderNumber-1)}", f"/content/{str(FolderNumber)}", "MelBand Roformer | Aspiration by Sucial", output_format, segment_size, overlap, batch_size, override_model_segment_size, use_autocast, extensions)
  for file1 in os.listdir(f"/content/{str(FolderNumber)}"):
    if("(aspiration)_aspiration_mel_band_roformer_sdr_18" in file1):
      print(f"removing: {file1}")
      !rm "/content/{FolderNumber}/{file1}"
  FolderNumber = FolderNumber + 1
  FolderNumberPrevious = FolderNumber - 1
  for file1 in os.listdir(f"/content/{str(FolderNumberPrevious)}"):
    !mv "/content/{FolderNumberPrevious}/{file1}" "/content/{FolderNumberPrevious}/{Name_Of_Audio}"
  if not os.path.exists(output_folder):
    os.makedirs(output_folder)
  FolderNumberPrevious = FolderNumber - 1
  for file1 in os.listdir(f"/content/{str(FolderNumberPrevious)}"):
    !cp "/content/{FolderNumberPrevious}/{file1}" "{output_folder}/{Name_Of_Audio_Fixed_NoHiss}"




#Saves the result and removes the hissing AND aggressively removes sound effects:
if Final_Aggressive_Mode == "MelBand Roformer | Karaoke by Gabox (Normal Aggression)":
  if os.path.exists(f"/content/{str(FolderNumber)}"):
    !rm -rf "/content/{FolderNumber}"
  uvr_cli(f"/content/{str(FolderNumber-1)}", f"/content/{str(FolderNumber)}", "MelBand Roformer | Karaoke by Gabox", output_format, segment_size, overlap, batch_size, override_model_segment_size, use_autocast, extensions)
  for file1 in os.listdir(f"/content/{str(FolderNumber)}"):
    if("(Instrumental)_mel_band_roformer_karaoke_aufr33_viperx_sdr_10" in file1):
      print(f"removing: {file1}")
      !rm "/content/{FolderNumber}/{file1}"
  FolderNumber = FolderNumber + 1
  FolderNumberPrevious = FolderNumber - 1
  for file1 in os.listdir(f"/content/{str(FolderNumberPrevious)}"):
    !mv "/content/{FolderNumberPrevious}/{file1}" "/content/{FolderNumberPrevious}/{Name_Of_Audio}"
  if not os.path.exists(output_folder):
    os.makedirs(output_folder)
  FolderNumberPrevious = FolderNumber - 1
  for file1 in os.listdir(f"/content/{str(FolderNumberPrevious)}"):
    !cp "/content/{FolderNumberPrevious}/{file1}" "{output_folder}/{Name_Of_Audio_Fixed_Aggressive_NoHiss}"


if Final_Aggressive_Mode == "UVR-MDX-NET_Crowd_HQ_1.onnx (EXTREMELY AGGRESSIVE)":
  if os.path.exists(f"/content/{str(FolderNumber)}"):
    !rm -rf "/content/{FolderNumber}"
  uvr_cli_MDX_NET(f"/content/{str(FolderNumber-1)}", f"/content/{str(FolderNumber)}", extensions, output_format, "UVR-MDX-NET_Crowd_HQ_1.onnx", hop_length, segment_size, denoise, overlap_2, batch_size, use_autocast)
  for file1 in os.listdir(f"/content/{str(FolderNumber)}"):
    if("(Crowd)_UVR-MDX-NET_Crowd_HQ_1" in file1):
      print(f"removing: {file1}")
      !rm "/content/{FolderNumber}/{file1}"
  FolderNumber = FolderNumber + 1
  FolderNumberPrevious = FolderNumber - 1
  for file1 in os.listdir(f"/content/{str(FolderNumberPrevious)}"):
    !mv "/content/{FolderNumberPrevious}/{file1}" "/content/{FolderNumberPrevious}/{Name_Of_Audio}"
  if not os.path.exists(output_folder):
    os.makedirs(output_folder)
  FolderNumberPrevious = FolderNumber - 1
  for file1 in os.listdir(f"/content/{str(FolderNumberPrevious)}"):
    !cp "/content/{FolderNumberPrevious}/{file1}" "{output_folder}/{Name_Of_Audio_Fixed_EXTREMELYAGGRESSIVE_NoHiss}"



#Saves the result and aggressively removes sound effects (hiss still there):
if Final_Aggressive_Mode == "MelBand Roformer | Karaoke by Gabox (Normal Aggression)":
  if os.path.exists(f"/content/{str(FolderNumber)}"):
    !rm -rf "/content/{FolderNumber}"
  uvr_cli(f"/content/{str(FolderNumber-3)}", f"/content/{str(FolderNumber)}", "MelBand Roformer | Karaoke by Gabox", output_format, segment_size, overlap, batch_size, override_model_segment_size, use_autocast, extensions)
  for file1 in os.listdir(f"/content/{str(FolderNumber)}"):
    if("(Instrumental)_mel_band_roformer_karaoke_aufr33_viperx_sdr_10" in file1):
      print(f"removing: {file1}")
      !rm "/content/{FolderNumber}/{file1}"
  FolderNumber = FolderNumber + 1
  FolderNumberPrevious = FolderNumber - 1
  for file1 in os.listdir(f"/content/{str(FolderNumberPrevious)}"):
    !mv "/content/{FolderNumberPrevious}/{file1}" "/content/{FolderNumberPrevious}/{Name_Of_Audio}"
  if not os.path.exists(output_folder):
    os.makedirs(output_folder)
  FolderNumberPrevious = FolderNumber - 1
  for file1 in os.listdir(f"/content/{str(FolderNumberPrevious)}"):
    !cp "/content/{FolderNumberPrevious}/{file1}" "{output_folder}/{Name_Of_Audio_Fixed_Aggressive}"



if Final_Aggressive_Mode == "UVR-MDX-NET_Crowd_HQ_1.onnx (EXTREMELY AGGRESSIVE)":
  if os.path.exists(f"/content/{str(FolderNumber)}"):
    !rm -rf "/content/{FolderNumber}"
  uvr_cli_MDX_NET(f"/content/{str(FolderNumber-3)}", f"/content/{str(FolderNumber)}", extensions, output_format, "UVR-MDX-NET_Crowd_HQ_1.onnx", hop_length, segment_size, denoise, overlap_2, batch_size, use_autocast)
  for file1 in os.listdir(f"/content/{str(FolderNumber)}"):
    if("(Crowd)_UVR-MDX-NET_Crowd_HQ_1" in file1):
      print(f"removing: {file1}")
      !rm "/content/{FolderNumber}/{file1}"
  FolderNumber = FolderNumber + 1
  FolderNumberPrevious = FolderNumber - 1
  for file1 in os.listdir(f"/content/{str(FolderNumberPrevious)}"):
    !mv "/content/{FolderNumberPrevious}/{file1}" "/content/{FolderNumberPrevious}/{Name_Of_Audio}"
  if not os.path.exists(output_folder):
    os.makedirs(output_folder)
  FolderNumberPrevious = FolderNumber - 1
  for file1 in os.listdir(f"/content/{str(FolderNumberPrevious)}"):
    !cp "/content/{FolderNumberPrevious}/{file1}" "{output_folder}/{Name_Of_Audio_Fixed_EXTREMELYAGGRESSIVE}"






print()
print()
print()
print("FINISHED!!! Check Google Drive for the output!")